In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Format_data").getOrCreate()

23/10/26 20:56:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# from minio.error import S3Error
from minio import Minio

minio_url = 'minio:9000'
access_key = "Qm6pkTNzxbz3Km6QHVwc"
secret_key = "wa3LjHTKv3kQOEg8SQjSivwKinh6PKRVZlnYkUbY"

minio_client = Minio(endpoint=minio_url,
                access_key=access_key,
                secret_key=secret_key,
                secure=False)

b_name = 'warehouse'
o_name = 'bronze/tracks/spotify_tracks.csv'
f_name = 'spotify_tracks.csv'

try:
    minio_client.fget_object(b_name, o_name, f_name)
finally:
    pass

In [3]:
df = spark.read.option("header",True).csv(f_name)

In [4]:
from pyspark.sql.functions import col, regexp_replace
# Sử dụng hàm regexp_replace để xử lý dữ liệu trong cột artist_id
df = df.withColumn("artist_ids", regexp_replace(col("artist_ids"), r"[\[\]']|\s", ""))

In [1]:
from pyspark.sql.functions import split, col
from pyspark.sql.types import ArrayType, StringType

# Định nghĩa hàm UDF (User-Defined Function) để chuyển đổi chuỗi thành mảng
split_udf = split(col("artist_ids"), ", ")
# Đổi cột với kiểu dữ liệu là ArrayType
df2 = df.withColumn("artist_ids", split_udf.cast(ArrayType(StringType())))

NameError: name 'df' is not defined

In [6]:
# Drop Duplicates
df2 = df2.dropDuplicates()
# Drop Null Values
df2 = df2.na.drop()
# Lọc ra các dòng có cột release_date khác "0000" hoặc là giá trị null
df2 = df2.filter((col("release_date") != "0000"))

In [7]:
# Explode multiple values in artist ids
from pyspark.sql.functions import explode
df1 = df2.withColumn("artist_ids",explode(df2.artist_ids))

In [8]:
# Format colume type
df2 = df1.withColumn("disc_number",df1.disc_number.cast('int'))
df1 = df2
df2 = df1.withColumn("popularity",df1.popularity.cast('int'))
df1 = df2
df2 = df1.withColumn("acousticness",df1.acousticness.cast('float'))
df1 = df2
df2 = df1.withColumn("danceability",df1.danceability.cast('float'))
df1 = df2
df2 = df1.withColumn("duration_ms",df1.duration_ms.cast('int'))
df1 = df2
df2 = df1.withColumn("energy",df1.energy.cast('float'))
df1 = df2
df2 = df1.withColumn("instrumentalness",df1.instrumentalness.cast('float'))
df1 = df2
df2 = df1.withColumn("key",df1.key.cast('int'))
df1 = df2
df2 = df1.withColumn("liveness",df1.liveness.cast('float'))
df1 = df2
df2 = df1.withColumn("loudness",df1.loudness.cast('float'))
df1 = df2
df2 = df1.withColumn("mode",df1.mode.cast('binary'))
df1 = df2
df2 = df1.withColumn("speechiness",df1.speechiness.cast('float'))
df1 = df2
df2 = df1.withColumn("tempo",df1.tempo.cast('float'))
df1 = df2
df2 = df1.withColumn("valence",df1.valence.cast('float'))
df1 = df2
df2 = df1.withColumn("release_date",df1.release_date.cast('date'))

In [16]:
# df2.write.saveAsTable('silver.spotify_tracks')

In [9]:
%%sql
CREATE DATABASE IF NOT EXISTS silver

23/10/19 13:14:02 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [3]:
# %%sql
# DROP DATABASE IF EXISTS silver CASCADE

++
||
++
++

In [10]:
%%sql
DROP TABLE IF EXISTS silver.spotify_tracks

++
||
++
++

In [11]:
%%sql
CREATE TABLE IF NOT EXISTS silver.spotify_tracks(
    track_id	              string,
    track_name            string,
    release_date      date,
    release_date_precision  string,
    album_id     string,
    artist_ids  string,
    disc_number         float,
    explicit             string,
    external_urls            string,
    popularity  float,
    acousticness float,
    danceability float,
    duration_ms int,
    energy float,
    instrumentalness float,
    key int,
    liveness float,
    loudness float,
    mode binary,
    speechiness float,
    tempo float,
    valence float
    )
USING iceberg
PARTITIONED BY (years(release_date))

++
||
++
++

In [12]:
# df2.writeTo("silver.spotify_tracks").partitionedBy("release_date").createOrReplace()
df2.writeTo("silver.spotify_tracks").append()

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".    (0 + 1) / 1]
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


In [13]:
%%sql
SHOW DATABASES;

namespace
silver


In [5]:
%%sql
SELECT * FROM silver.spotify_tracks ORDER BY release_date DESC

track_id,track_name,release_date,release_date_precision,album_id,artist_ids,disc_number,explicit,external_urls,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,valence
2djNsYsA71cIs4A8YaF59J,WHAT JUST HAPPENED,2023-10-27,day,5cFhETbBm8L6tybXY4AZSj,2tIP7SsRs7vjIcLrU85W8J,1.0,False,https://open.spotify.com/track/2djNsYsA71cIs4A8YaF59J,0.0,0.10899999737739563,0.5839999914169312,162191,0.5820000171661377,0.0,10,0.3230000138282776,-5.124000072479248,bytearray(b'1'),0.042899999767541885,170.99600219726562,0.47999998927116394
19bgct6U5djwkDhAttMUlb,Ai Lau Đôi Mi Hoen,2023-10-26,day,1dSX1ojYBuexp6ZaIaXJ9g,3XOuN4VDN4EL0GjKov5mWL,1.0,False,https://open.spotify.com/track/19bgct6U5djwkDhAttMUlb,0.0,0.14399999380111694,0.7120000123977661,215217,0.6959999799728394,3.6300000374467345e-06,5,0.16300000250339508,-6.5970001220703125,bytearray(b'0'),0.028300000354647636,114.98799896240234,0.5799999833106995
0rd8Db67ZWjyB9ihsKDtF8,HIT ME UP REMIX,2023-10-26,day,2f8vMKn3STdMNihwWcdP7C,2nSO7JYDbJrYbJmP39qUzj,1.0,False,https://open.spotify.com/track/0rd8Db67ZWjyB9ihsKDtF8,0.0,0.003700000001117587,0.7519999742507935,213980,0.8650000095367432,0.3840000033378601,9,0.12200000137090683,-6.409999847412109,bytearray(b'1'),0.04270000010728836,144.0469970703125,0.8870000243186951
2zaakMjyU4c631ynSxK8Wx,bướngg,2023-10-26,day,5W6ez1UTw9NXcbBfP4UmZE,"2rVYwIYNEsMxtt8j5yemHb,1QyEugFxmSpHLGfrQsRUor",1.0,False,https://open.spotify.com/track/2zaakMjyU4c631ynSxK8Wx,0.0,0.1420000046491623,0.8399999737739563,166000,0.4099999964237213,1.8999999156221747e-05,2,0.2639999985694885,-7.158999919891357,bytearray(b'1'),0.05090000107884407,89.99299621582031,0.8700000047683716
1TwRo03GTMwKviM9KFTGpK,Sunset In The City,2023-10-26,day,368ne0BxywURFHQ43RR0zf,6CGGvCBHWqQ4HXtn5aLhbh,1.0,False,https://open.spotify.com/track/1TwRo03GTMwKviM9KFTGpK,0.0,0.10899999737739563,0.7210000157356262,224100,0.48500001430511475,4.600000011123484e-06,2,0.1589999943971634,-8.729000091552734,bytearray(b'1'),0.0357000008225441,100.02300262451172,0.46299999952316284
5gUpRXPu9hECmXNUUs6BPo,HIT ME UP REMIX BEAT,2023-10-26,day,5z0MWeVORA4bKlPMlKs1fX,2nSO7JYDbJrYbJmP39qUzj,1.0,False,https://open.spotify.com/track/5gUpRXPu9hECmXNUUs6BPo,0.0,0.0002010000025620684,0.7689999938011169,213982,0.9100000262260437,0.781000018119812,9,0.12399999797344208,-7.425000190734863,bytearray(b'1'),0.05609999969601631,143.99600219726562,0.9710000157356262
3un8UkiPHJjlsrTHQnisXH,Từng Quen,2023-10-25,day,2M0vYJZddXn6qRpeVtGvDZ,"5M3ffmRiOX9Q8Y4jNeR5wu,0hy8mfXxfD8kQ6MRQkDVrn",1.0,False,https://open.spotify.com/track/3un8UkiPHJjlsrTHQnisXH,0.0,0.2029999941587448,0.7580000162124634,175545,0.2460000067949295,0.0,7,0.10100000351667404,-16.913000106811523,bytearray(b'1'),0.17000000178813934,77.01000213623047,0.49000000953674316
5MiIaWMDwGGWHY56oXRhlJ,Mưa Tháng Sáu,2023-10-24,day,2jNxIyiGLvQz7UPCRUuU76,"63SNH9m8M034lCGELVC1dm,6d0dLenjy5CnR5ZMn2agiV,5Cf9nBDNc99UFkq9Yqap7Q",1.0,False,https://open.spotify.com/track/5MiIaWMDwGGWHY56oXRhlJ,19.0,0.8009999990463257,0.6190000176429749,257143,0.4269999861717224,1.8800000134433503e-06,2,0.36800000071525574,-7.142000198364258,bytearray(b'0'),0.056299999356269836,139.7570037841797,0.2840000092983246
1HdleLfTsgdP0M0wEaS7zf,Bay Cùng Bay,2023-10-24,day,2jNxIyiGLvQz7UPCRUuU76,"63SNH9m8M034lCGELVC1dm,3Ufoo4BPShhahtCSjgpBLP,2dP2cG3zK05Y1jtb8Kwdpm",1.0,False,https://open.spotify.com/track/1HdleLfTsgdP0M0wEaS7zf,22.0,0.23199999332427979,0.41499999165534973,252952,0.3709999918937683,0.33799999952316284,0,0.1080000028014183,-8.626999855041504,bytearray(b'0'),0.03629999980330467,179.86399841308594,0.29499998688697815
0KnWLhf5GqPEC4m7sHhxrY,Cơn Mưa Rào,2023-10-24,day,2jNxIyiGLvQz7UPCRUuU76,"63SNH9m8M034lCGELVC1dm,4fg8aMZ23d2bxKz7r2vt9v,3Ufoo4BPShhahtCSjgpBLP",1.0,False,https://open.spotify.com/track/0KnWLhf5GqPEC4m7sHhxrY,22.0,0.5260000228881836,0.6949999928474426,225000,0.6869999766349792,0.0011099999537691474,3,0.16200000047683716,-5.520999